テスト用

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import catboost as cb
from matplotlib import pyplot as plt
import warnings
warnings.simplefilter('ignore')

In [282]:
data_dir = "F:/TARGET frontier/TXT35/"
data_name = "data"
tail = "fe3"
train_data = pd.DataFrame()
for i in range(2015,2018+1):
    res = pd.read_csv(data_dir+str(i)+data_name+tail+".csv",encoding="shift-jis")
    train_data = pd.concat([train_data,res])

#test_data
test_data = pd.read_csv(data_dir+"2019data"+tail+".csv",encoding="shift-jis")

In [283]:
train_data = train_data.query("馬場状態 != 1")
test_data = test_data.query("馬場状態 != 1")

In [284]:
#train_pred = train_data.copy()
train_data = train_data.query("単勝オッズ >= 5")

In [285]:
pd.set_option('display.max_columns', 100)
train_data.head()

,レースID(新),場所,芝・ダ,距離,クラス名,馬場状態,出走頭数,騎手,枠番,性別,年齢,キャリア,斤量,馬体重,馬体重増減,間隔,休み明け〜戦目,種牡馬,父タイプ名,所属,調教師,入線順位,確定着順,着差,単勝オッズ,複勝オッズ下限,複勝オッズ上限,複勝配当,場所距離芝ダ,前走レースID(新),前走場所,前芝・ダ,前距離,前クラス名,前走馬場状態,前走出走頭数,替,前騎手,前走枠番,前走脚質,前走斤量,前走Ave-3F,前走上り3F,前走上り3F順,前走PCI3,前走RPCI,前PCI,前好走,前走入線順位,前走確定着順,...,前走単勝オッズ,前走場所距離芝ダ,前走レースID(新)1,前走場所1,前芝・ダ1,前距離1,前クラス名1,前走馬場状態1,前走出走頭数1,替1,前騎手1,前走枠番1,前走脚質1,前走斤量1,前走Ave-3F1,前走上り3F1,前走上り3F順1,前走PCI31,前走RPCI1,前PCI1,前好走1,前走入線順位1,前走確定着順1,前走着差タイム1,前走単勝オッズ1,前走場所距離芝ダ1,前走レースID(新)2,前走場所2,前芝・ダ2,前距離2,前クラス名2,前走馬場状態2,前走出走頭数2,替2,前騎手2,前走枠番2,前走脚質2,前走斤量2,前走Ave-3F2,前走上り3F2,前走上り3F順2,前走PCI32,前走RPCI2,前PCI2,前好走2,前走入線順位2,前走確定着順2,前走着差タイム2,前走単勝オッズ2,前走場所距離芝ダ2
0,201512270605081112,2,2,1200,2,3,16,497,6,2,3,13,56.0,492,-4.0,6.0,5.0,675,8,0,363,1,1,0.0,5.4,1.9,2.7,190.0,76,201511140505031003,14,2,1400.0,2,2,15.0,0,333,2.0,3,54.0,37.2,33.9,10.0,59.53,58.2,59.7,0,8.0,8.0,...,14.6,55,201510100804011013,3.0,2.0,1400.0,2.0,3.0,18.0,0.0,435.0,7.0,2.0,53.0,35.03,34.0,11.0,54.30,51.8,53.0,0.0,8.0,8.0,0.3,10.1,202.0,201508220102011003,13.0,2.0,1200.0,1.0,3.0,16.0,0.0,101.0,2.0,2.0,54.0,34.1,34.8,3.0,47.77,44.6,48.0,0.0,1.0,1.0,0.0,15.3,44.0
2,201512270605081106,2,2,1200,2,3,16,248,3,2,3,9,56.0,478,6.0,5.0,2.0,714,5,1,241,3,3,0.2,6.2,1.8,2.5,180.0,76,201511210805051217,3,2,1200.0,2,3,18.0,0,161,8.0,2,56.0,34.5,34.4,9.0,50.13,48.0,50.3,0,7.0,7.0,...,28.6,197,201505170803081006,3.0,2.0,1200.0,12.0,3.0,16.0,0.0,455.0,3.0,2.0,56.0,34.80,34.1,10.0,51.90,50.0,52.1,0.0,9.0,9.0,0.9,118.0,197.0,201503220602081104,2.0,2.0,1800.0,7.0,3.0,12.0,0.0,358.0,4.0,5.0,56.0,37.3,37.3,11.0,60.53,58.1,50.0,1.0,11.0,11.0,2.8,327.2,122.0
3,201512270605081108,2,2,1200,2,3,16,395,4,1,5,17,55.0,484,-4.0,2.0,4.0,108,1,1,377,4,4,0.3,6.6,1.7,2.5,0.0,76,201512130605040911,2,2,1200.0,2,2,16.0,1,395,6.0,2,53.0,34.1,34.4,3.0,48.17,46.3,49.1,0,3.0,3.0,...,19.1,118,201510240804060915,3.0,2.0,1200.0,2.0,3.0,15.0,0.0,266.0,8.0,3.0,55.0,34.30,34.7,15.0,52.50,51.2,48.8,1.0,12.0,12.0,1.4,21.5,197.0,201509200604041005,2.0,2.0,1200.0,2.0,3.0,16.0,0.0,395.0,3.0,2.0,55.0,34.4,34.4,5.0,50.80,47.1,50.0,0.0,4.0,4.0,0.2,12.5,118.0
4,201512270605081116,2,2,1200,2,3,16,449,8,1,3,15,54.0,440,-6.0,2.0,4.0,393,4,1,186,5,5,0.3,33.9,6.3,9.7,0.0,76,201512130605040904,2,2,1200.0,2,2,16.0,0,252,2.0,5,53.0,33.6,36.4,15.0,48.17,46.3,42.3,1,13.0,13.0,...,8.6,118,201510310303011107,19.0,2.0,1200.0,2.0,3.0,16.0,1.0,252.0,4.0,5.0,51.0,33.90,34.3,8.0,51.30,49.1,48.8,0.0,2.0,2.0,0.1,20.8,96.0,201509270604070708,2.0,2.0,1200.0,1.0,2.0,16.0,0.0,121.0,4.0,3.0,50.0,34.6,34.3,8.0,51.30,49.7,50.9,0.0,1.0,1.0,0.0,59.4,118.0
5,201512270605081110,2,2,1200,2,3,16,317,5,2,4,16,57.0,464,4.0,8.0,2.0,400,6,1,223,6,6,0.4,12.2,2.8,4.1,0.0,76,201510310303011116,19,2,1200.0,2,3,16.0,0,103,8.0,2,57.0,34.4,34.6,12.0,51.30,49.1,49.4,0,14.0,14.0,...,12.9,96,201503011001081204,10.0,2.0,1200.0,2.0,4.0,18.0,1.0,103.0,2.0,3.0,55.0,34.60,35.0,2.0,48.23,47.2,48.9,0.0,1.0,1.0,0.0,6.2,2.0,201502071001010909,10.0,2.0,1200.0,1.0,2.0,18.0,1.0,103.0,5.0,3.0,57.0,33.3,34.4,9.0,46.90,44.5,46.8,0.0,1.0,1.0,0.0,2.8,2.0


In [286]:
columns_name = ['芝・ダ', '距離', '馬場状態','クラス名', '騎手', '枠番', '性別', '年齢', 'キャリア',
       '斤量', '間隔', '休み明け〜戦目', '種牡馬', '前芝・ダ', '前距離',
       '前クラス名', '前走馬場状態','前騎手', '前走枠番', '前走脚質', '前走斤量', '前走Ave-3F',
       '前走上り3F', '前好走', '前走着差タイム',"単勝オッズ",
       '前走単勝オッズ',  '出走頭数', '場所距離芝ダ', '前走場所距離芝ダ']
columns_name += ['前走着差タイム1','前走着差タイム2']
cat_columns = [ '芝・ダ','クラス名',  '騎手', '性別', '馬場状態','前走馬場状態',
               '種牡馬', '前芝・ダ',
       '前クラス名', '前騎手','前走脚質', '前好走', 
       '場所距離芝ダ', '前走場所距離芝ダ']

In [287]:
use_train_columns = columns_name
cat_feature = cat_columns

In [288]:
train_x = train_data[use_train_columns]
test_x = test_data[use_train_columns]

In [289]:
test_y = test_data["入線順位"]
train_y = train_data["入線順位"]

In [290]:
#カテゴリカル変数になっている特徴量の列名を指定
categorical_features_indices = cat_feature

In [291]:
#これがないとエラー履くので注意
train_y.fillna(7,inplace=True) #無ければ10位に
test_y.fillna(7,inplace=True)

In [292]:
#上記のものは全て文字列またはint型である必要があり、floatや欠損値は変換しなければならない
for i in categorical_features_indices:
    train_x[i].fillna(9999,inplace=True)
    test_x[i].fillna(9999,inplace=True)
for i in categorical_features_indices:
    if train_x[i].dtype != 'int64':
        train_x[i] = train_x[i].astype('int64')
    if test_x[i].dtype != 'int64':
        test_x[i] = test_x[i].astype('int64')

In [293]:
scores = []
histories = []
models = []
y = pd.Series()
kf = KFold(n_splits=4,shuffle=False)
for tr_idx,va_idx in kf.split(train_x):
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    ############
    """print(tr_x.shape)
    tr_x, va_x = tr_x.query("単勝オッズ >= 10"), va_x.query("単勝オッズ >= 10")
    tr_y, va_y = tr_y.query("単勝オッズ >= 10"), va_y.query("単勝オッズ >= 10")"""
    ############
    evals_result = {}
    model = cb.CatBoostRegressor(iterations=10000,
                                 learning_rate=0.01,
                                 depth=6,
                                 verbose=100,
                                 use_best_model=True,
                                 early_stopping_rounds=100,
                                 one_hot_max_size=100,
                                 task_type="GPU",
                                 devices='0:1'
                                 
                                )
    
    model.fit(tr_x,
              tr_y,
              cat_features=categorical_features_indices,
              eval_set=(va_x, va_y),
              plot=False,

             )
    models.append(model)
    #histories.append(evals_result)
    va_pred = model.predict(va_x)
    va_y.fillna(0.5,inplace=True)
    score = mean_squared_error(va_y,va_pred)
    score = np.sqrt(mean_squared_error(va_y,va_pred)) #rmse
    scores.append(score)
    
    #evaluationさせる
    #全部のfoldについてyに入れておいて、
    y_pred_proba = model.predict(va_x)
    y = pd.concat([y,pd.Series(y_pred_proba)])


output = train_data
output = output.assign(ya=y.values)
output["複勝配当"] = output["複勝配当"].astype("int64")
output.to_csv("C:/programs/HRP/kfeval2.csv",index=False,encoding="shift-jis")
    
print(*scores)
print(np.mean(scores))

0:	learn: 4.3078300	test: 4.3403488	best: 4.3403488 (0)	total: 33.8ms	remaining: 5m 37s
100:	learn: 3.7454406	test: 3.7865833	best: 3.7865833 (100)	total: 3.08s	remaining: 5m 1s
200:	learn: 3.6314821	test: 3.6741829	best: 3.6741829 (200)	total: 6.24s	remaining: 5m 4s
300:	learn: 3.5956346	test: 3.6405248	best: 3.6405248 (300)	total: 9.23s	remaining: 4m 57s
400:	learn: 3.5731024	test: 3.6225216	best: 3.6225216 (400)	total: 12.7s	remaining: 5m 3s
500:	learn: 3.5567339	test: 3.6113652	best: 3.6113652 (500)	total: 15.7s	remaining: 4m 57s
600:	learn: 3.5448613	test: 3.6044143	best: 3.6044143 (600)	total: 18.8s	remaining: 4m 53s
700:	learn: 3.5362593	test: 3.6000166	best: 3.6000166 (700)	total: 22s	remaining: 4m 52s
800:	learn: 3.5290740	test: 3.5967110	best: 3.5967110 (800)	total: 25.3s	remaining: 4m 50s
900:	learn: 3.5232525	test: 3.5944789	best: 3.5944789 (900)	total: 28.4s	remaining: 4m 47s
1000:	learn: 3.5182284	test: 3.5928562	best: 3.5928562 (1000)	total: 31.9s	remaining: 4m 46s
1100:

## 学習結果
1.xg0.0.8再現
loss:3.5029
1~3位PHR: 1.593
1~3位WHR: 0.637
1~6位PHR: 2.401
回収率:119% (128%)

2.前走上がり3F順位,前着差タイム1.2追加
3.4985
1~3位PHR: 1.587
1~3位WHR: 0.639
1~6位PHR: 2.401

3.場所、前走場所、父タイプカット(1派生)
loss:3.5026
1~3位PHR: 1.595
1~3位WHR: 0.638
1~6位PHR: 2.399
回収率:153.10% (114%)

4.PCI系カット(3派生)
loss:3.50323
1~3位PHR: 1.593
1~3位WHR: 0.638
1~6位PHR: 2.400
回収率:142.89% (130%)

5.(4派生)馬場状態全カット、前2,3走の着差入れ
loss:3.5005
1~3位PHR: 1.592
1~3位WHR: 0.638
1~6位PHR: 2.399
回収率:154.79% (110%)

6.4派生,調教師カット
loss:3.5024
1~3位PHR: 1.581
1~3位WHR: 0.639
1~6位PHR: 2.400
回収率:150.32% (80%)


In [294]:
test_length = len(test_data)
predict = np.array([0.0]*test_length)
for i in range(4):
    y_pred_proba = models[i].predict(test_x)
    predict += y_pred_proba

predict /= 4

In [295]:
#y = pd.DataFrame(predict,columns=["y'"])
output = test_data #ここも書き換えること

output["ya"] = predict
output.head()
output["複勝配当"] = output["複勝配当"].astype("int64")
output.to_csv("C:/programs/HRP/neweval11.csv",index=False,encoding="shift-jis")

In [136]:
fscore = model.get_feature_importance()

In [137]:
jscore = []
for i,j in zip(use_train_columns,fscore):
    jscore.append((i,np.round(j,decimals=3)))
    

In [138]:
from operator import itemgetter
jscore.sort(reverse=True,key=itemgetter(1))

In [139]:
jscore

[('単勝オッズ', 59.355),
 ('出走頭数', 19.564),
 ('前走着差タイム', 2.191),
 ('前走脚質', 2.07),
 ('間隔', 1.814),
 ('クラス名', 1.719),
 ('前クラス名', 1.386),
 ('前芝・ダ', 1.264),
 ('前距離', 1.231),
 ('前走場所距離芝ダ', 1.163),
 ('前好走', 1.156),
 ('キャリア', 1.07),
 ('芝・ダ', 0.945),
 ('休み明け〜戦目', 0.854),
 ('場所距離芝ダ', 0.665),
 ('年齢', 0.562),
 ('前走Ave-3F', 0.509),
 ('種牡馬', 0.498),
 ('調教師', 0.465),
 ('枠番', 0.306),
 ('騎手', 0.292),
 ('距離', 0.2),
 ('性別', 0.19),
 ('前走上り3F', 0.162),
 ('前走単勝オッズ', 0.156),
 ('斤量', 0.149),
 ('前走斤量', 0.059),
 ('前騎手', 0.004),
 ('馬場状態', 0.0),
 ('前走馬場状態', 0.0),
 ('前走枠番', 0.0)]

In [ ]:
slen = 20
label = []
weight = []
for i,j in fscore[:slen]:
    label.append(i)
    weight.append(j)

plt.rcdefaults() #横棒グラフモード
fig, ax = plt.subplots()
y_pos = np.arange(slen)
ax.barh(y_pos, weight,align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(label)
ax.invert_yaxis()
ax.set_xlabel('label')
ax.set_title('xgboost特徴量')
plt.show()
    

In [ ]:
#save models
import pickle
with open('xgmachina_ver0.0.7.pickle', mode='wb') as fp:
    pickle.dump(models, fp)

In [ ]:
#import models
"""import pickle
with open('xgmachina_ver0.0.6.pickle', mode='rb') as fp:
    models = pickle.load(fp)"""